In [29]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [30]:
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'top_model_weight.h5'
# Save the checkpoint in the /model_weight folder
filepath = 'model_weight/My_Model_Weight.hdf5'
# dimensions of our images.
img_width, img_height = 224, 224
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'
nb_train_samples = 1620
nb_validation_samples = 370
nb_test_samples = 88
epochs = 5
batch_size = 10

In [31]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
#base_model.load_weights(weights_path)
print('Model loaded.')

Model loaded.


In [32]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))
top_model.save_weights('top_model_weight.h5')

In [33]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier, in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base model
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
sgd = SGD(1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile( loss='categorical_crossentropy',
               optimizer=sgd, 
               metrics=['accuracy'] )

In [34]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

model.summary()

Found 1620 images belonging to 3 classes.
Found 370 images belonging to 3 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [35]:
model.load_weights('My_Model_Weight.h5')


In [41]:

# Keep only a single checkpoint, the best over test accuracy.
checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

In [42]:
# load the model's weights
model.load_weights(filepath)

In [43]:
# fine-tune the model
history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1,callbacks=[checkpoint])



Epoch 1/10
162/162 [==============================] - 4695s 29s/step - loss: 0.5970 - acc: 0.6426 - val_loss: 0.6869 - val_acc: 0.6081

Epoch 00001: val_acc improved from -inf to 0.60811, saving model to model_weight/My_Model_Weight.hdf5
Epoch 2/10
162/162 [==============================] - 4618s 29s/step - loss: 0.5888 - acc: 0.6395 - val_loss: 0.6807 - val_acc: 0.6297

Epoch 00002: val_acc improved from 0.60811 to 0.62973, saving model to model_weight/My_Model_Weight.hdf5
Epoch 3/10
162/162 [==============================] - 4601s 28s/step - loss: 0.5743 - acc: 0.6407 - val_loss: 0.6317 - val_acc: 0.6378

Epoch 00003: val_acc improved from 0.62973 to 0.63784, saving model to model_weight/My_Model_Weight.hdf5
Epoch 4/10
162/162 [==============================] - 4573s 28s/step - loss: 0.5886 - acc: 0.6278 - val_loss: 0.6224 - val_acc: 0.6432

Epoch 00004: val_acc improved from 0.63784 to 0.64324, saving model to model_weight/My_Model_Weight.hdf5
Epoch 5/10
162/162 [===================

In [45]:
model.save_weights('My_Model_Weight.h5')